# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import api_google

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("../PyWeather/city_df.csv")
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.60,-73.97,42.01,70,100,3.80,US,1605233885
1,Rikitea,-23.12,-134.97,74.66,74,73,15.41,PF,1605233915
2,Narrabri,-30.32,149.78,82.40,37,0,26.40,AU,1605234094
3,Hong Kong,22.29,114.16,75.99,38,0,11.41,HK,1605233843
4,Ribeira Grande,38.52,-28.70,63.34,81,3,3.18,PT,1605234096
...,...,...,...,...,...,...,...,...,...
539,Bilma,18.69,12.92,69.17,31,0,8.81,NE,1605234331
540,Batavia,41.85,-88.31,46.99,70,90,5.82,US,1605234309
541,Ndjolé,-0.18,10.76,73.42,96,100,0.51,GA,1605234331
542,Anávra,39.20,22.08,55.99,93,40,1.59,GR,1605234332


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
ideal_weather_df = city_df.loc[(city_df["Max Temp"]>50) & (city_df["Max Temp"]<80) & (city_df["Cloudiness"]>80)]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,Puerto Ayora,-0.74,-90.35,68.00,95,97,3.00,EC,1605233929
23,Chaman,30.92,66.45,56.75,38,86,5.17,PK,1605234103
25,Camopi,3.17,-52.32,74.64,93,96,3.31,GF,1605234105
27,Busselton,-33.65,115.33,66.00,61,100,5.99,AU,1605233913
36,Carutapera,-1.20,-46.02,78.66,86,89,5.50,BR,1605234111


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df[["City", "Lat", "Lng", "Hotel Name", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]


In [43]:
hotel_df.head()

,City,Lat,Lng,Hotel Name,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,Puerto Ayora,-0.74,-90.35,,68.00,95,97,3.00,EC,1605233929
23,Chaman,30.92,66.45,,56.75,38,86,5.17,PK,1605234103
25,Camopi,3.17,-52.32,,74.64,93,96,3.31,GF,1605234105
27,Busselton,-33.65,115.33,,66.00,61,100,5.99,AU,1605233913
36,Carutapera,-1.20,-46.02,,78.66,86,89,5.50,BR,1605234111


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
